## convert_cityscapes_to_coco

In [3]:
#!/usr/bin/env python3
# convert_cityscapes_to_coco_fixed.py
#
# Reads dataset/{train,val}/img + dataset/{train,val}/mask (PNG color‐coded)
# and produces dataset/train_coco.json & dataset/val_coco.json in COCO format,
# with live progress, timing estimates, and proper JSON‐serializable RLE.

import os
import json
import time
import numpy as np
from PIL import Image
from pycocotools import mask as maskUtils
from tqdm import tqdm

# 1) Adjust these paths if needed
DATA_ROOT = "dataset"
SPLITS    = ["train", "val"]

# 2) Your merged class names & color→class map
CLASS_NAMES = [
    "road_sidewalk_parking",
    "traffic",
    "person",
    "vehicle",
    "pole",
    "structure",
    "ground",
    "dynamic",
    "terrain",
    "sky",
]
PALETTE = {
    (128,64,128):0, (244,35,232):0, (250,170,160):0,
    (250,170,30):1, (220,220,0):1,
    (220,20,60):2, (255,0,0):2,
    (0,0,142):3, (0,0,70):3, (0,60,100):3, (0,0,90):3,
    (0,0,110):3,(0,80,100):3,(0,0,230):3,(119,11,32):3,
    (153,153,153):4,
    (70,70,70):5,(102,102,156):5,(190,153,153):5,
    (180,165,180):5,(150,100,100):5,
    (81,0,81):6, (111,74,0):7, (152,251,152):8, (70,130,180):9,
}

def convert_split(split):
    img_dir  = os.path.join(DATA_ROOT, split, "img")
    mask_dir = os.path.join(DATA_ROOT, split, "mask")
    out_json = os.path.join(DATA_ROOT, f"{split}_coco.json")

    coco = {"images": [], "annotations": [], "categories": []}
    for cid, name in enumerate(CLASS_NAMES):
        coco["categories"].append({"id": cid, "name": name})

    img_files = sorted([
        f for f in os.listdir(img_dir)
        if f.lower().endswith((".png",".jpg",".jpeg"))
    ])
    total = len(img_files)
    ann_id = 1
    img_id = 1
    start_time = time.time()

    for fn in tqdm(img_files, desc=f"[{split}]", unit="img"):
        # 1) Image entry
        img_path = os.path.join(img_dir, fn)
        with Image.open(img_path) as im:
            w, h = im.size
        coco["images"].append({
            "id": img_id, "file_name": fn, "width": w, "height": h
        })

        # 2) Load & process mask
        m = np.array(Image.open(os.path.join(mask_dir, fn)).convert("RGB"))
        for rgb, cid in PALETTE.items():
            comp = (m == rgb).all(axis=-1).astype(np.uint8)
            if comp.sum() == 0:
                continue
            rle = maskUtils.encode(np.asfortranarray(comp))
            # convert bytes → str for JSON
            rle["counts"] = rle["counts"].decode("ascii")

            area = int(comp.sum())
            bbox = maskUtils.toBbox(rle).tolist()
            coco["annotations"].append({
                "id": ann_id,
                "image_id": img_id,
                "category_id": cid,
                "segmentation": rle,
                "area": area,
                "bbox": bbox,
                "iscrowd": 0
            })
            ann_id += 1

        img_id += 1

        # ETA every 100 images
        if img_id % 100 == 0 or img_id == total:
            elapsed = time.time() - start_time
            per_img = elapsed / img_id
            eta = per_img * (total - img_id)
            print(f"  [{split}] {img_id}/{total} — elapsed {elapsed:.1f}s, ETA {eta:.1f}s")

    # 3) Write JSON
    with open(out_json, "w") as f:
        json.dump(coco, f)
    total_time = time.time() - start_time
    print(f"\nFinished {split}: {total} images, {ann_id-1} annotations in {total_time:.1f}s")
    print(f"Saved: {out_json}\n")

if __name__ == "__main__":
    for split in SPLITS:
        convert_split(split)


[train]:   3%|▎         | 99/2975 [01:25<52:08,  1.09s/img] 

  [train] 100/2975 — elapsed 85.7s, ETA 2463.5s


[train]:   7%|▋         | 199/2975 [03:27<44:48,  1.03img/s]  

  [train] 200/2975 — elapsed 207.4s, ETA 2877.4s


[train]:  10%|█         | 299/2975 [04:59<34:55,  1.28img/s]

...

  [val] 400/500 — elapsed 364.1s, ETA 91.0s


[val]: 100%|█████████▉| 499/500 [07:20<00:00,  1.01img/s]

  [val] 500/500 — elapsed 440.9s, ETA 0.0s


[val]: 100%|██████████| 500/500 [07:22<00:00,  1.13img/s]



Finished val: 500 images, 6073 annotations in 445.0s
Saved: dataset\val_coco.json



In [4]:
#!/usr/bin/env python3
# sample_coco_subset.py

import json, random, os

IN_JSON   = "dataset/train_coco.json"
OUT_JSON  = "dataset/train_coco_small.json"
N_SAMPLES = 500   # change as you like

with open(IN_JSON) as f:
    coco = json.load(f)

# pick random image IDs
all_imgs = coco["images"]
picked   = random.sample(all_imgs, min(N_SAMPLES, len(all_imgs)))
picked_ids = {img["id"] for img in picked}

# filter annotations to those images
anns = [ann for ann in coco["annotations"] if ann["image_id"] in picked_ids]

small = {
    "images": picked,
    "annotations": anns,
    "categories": coco["categories"]
}

with open(OUT_JSON, "w") as f:
    json.dump(small, f)
print(f"Sampled {len(picked)} images → {OUT_JSON}")


Sampled 500 images → dataset/train_coco_small.json


## yolo V1

In [5]:
#!/usr/bin/env python3
# train_yolov8_seg.py
#
# Full pipeline for YOLOv8-segmentation on your COCO‐format Cityscapes data.
# Uses medium model for speed+power, FP16, cosine LR, early stopping.

import os
from ultralytics import YOLO

def main():
    # Paths
    DATA_YAML = 'data.yaml'       # points to train/val COCO JSONs and class names
    PROJECT   = 'runs/seg'        # where to save experiments
    NAME      = 'cityscapes_yolov8m_seg'

    # Instantiate a medium‐size segmentation model
    model = YOLO('yolov8m-seg.pt')  # alternatives: yolov8n-seg.pt (nano), yolov8s-seg.pt (small)

    # Train with advanced settings
    model.train(
        data=DATA_YAML,
        epochs=100,             # train up to 100 epochs
        batch=8,                # batch size (adjust to your CPU RAM)
        imgsz=1024,             # input resolution (Square: 1024×1024 for best Cityscapes detail)
        device='cpu',           # CPU training
        workers=4,              # parallel data loaders
        optimizer='AdamW',      # more stable than SGD on small datasets
        lr0=1e-3,               # initial learning rate
        lrf=0.01,               # final lr is lr0 * lrf (cosine decay)
        augment=True,           # built-in mosaic, mixup, etc.
        val=True,               # run validation each epoch
        patience=10,            # early stopping after 10 epochs without val mAP75 improvement
        save_period=5,          # save a checkpoint every 5 epochs
        save=True,              # save the best model automatically
        project=PROJECT,
        name=NAME,
        exist_ok=True,          # overwrite existing runs/seg/NAME
        fp16=True               # enable mixed-precision on CPU (if supported)
    )

    print(f"\n✅ Training complete. Checkpoint & logs in {os.path.join(PROJECT,NAME)}")

if __name__ == '__main__':
    main()


  0%|          | 0.00/52.4M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
#!/usr/bin/env python3
# train_yolov8_seg_fast.py

from ultralytics import YOLO

def main():
    DATA_YAML = "data.yaml"          # in your project root
    MODEL     = "yolov8n-seg.pt"
    EPOCHS    = 20
    BATCH     = 2
    IMGSZ     = 512
    PATIENCE  = 5
    PROJECT   = "runs/seg"
    NAME      = "cityscapes_yolov8n_seg_fast"

    model = YOLO(MODEL)

    model.train(
        data=DATA_YAML,      # ← your data.yaml
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=IMGSZ,
        device="cpu",
        workers=3,
        patience=PATIENCE,
        project=PROJECT,
        name=NAME,
        exist_ok=True,
        optimizer="AdamW",
        lr0=1e-3,
        lrf=0.05,
        augment=True,
        val=True,
        save=True,
        half=False         # ← use half FP32/FP16 toggle instead of fp16=
    )

    print(f"\n✅ Done! Checkpoint & logs in {PROJECT}/{NAME}")

if __name__ == "__main__":
    main()


New https://pypi.org/project/ultralytics/8.3.127 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.78  Python-3.8.7 torch-1.13.1+cpu CPU (Intel Pentium Silver N5030 1.10GHz)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=data.yaml, epochs=30, time=None, patience=5, batch=2, imgsz=512, save=True, save_period=-1, cache=False, device=cpu, workers=2, project=runs/seg, name=cityscapes_yolov8n_seg_fast, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nm

train: Scanning C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\train\img... 0 images, 2975 backgrounds, 0 corrupt: 100%|██████████| 2975/2975 [00:04<00:00, 731.82it/s]

train: WARNING  No labels found in C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\train\img.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


train: New cache created: C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\train\img.cache
WARNING  No labels found in C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\train\img.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


c:\Users\MBQ\AppData\Local\Programs\Python\Python38\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.6 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\val\img... 0 images, 500 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 732.49it/s]

val: WARNING  No labels found in C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\val\img.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


val: New cache created: C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\val\img.cache
WARNING  No labels found in C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\val\img.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs\seg\cityscapes_yolov8n_seg_fast\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 512 train, 512 val
Using 0 dataloader workers
Logging results to runs\seg\cityscapes_yolov8n_seg_fast
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/30         0G          0          0      2.515          0          0        512:   2%|▏         | 31/1488 [02:04<1:37:29,  4.01s/it]


KeyboardInterrupt: 

In [11]:
#!/usr/bin/env python3
# train_yolov8_seg_fast.py  (v2)

from ultralytics import YOLO

def main():
    DATA_YAML = "data.yaml"
    MODEL     = "yolov8n-seg.pt"
    PROJECT   = "runs/seg"
    NAME      = "cityscapes_yolov8n_seg_faster"

    # these hyperparams are tuned for CPU speed + decent quality
    model = YOLO(MODEL)
    model.train(
        data      = DATA_YAML,
        epochs    = 20,          # early stopping will cut this short
        batch     = 2,           # keep small so you don't OOM
        imgsz     = 384,         # down from 512 → much faster
        device    = "cpu",
        workers   = 4,           # match your CPU cores
        cache     = True,        # cache dataset in RAM/disk
        freeze    = [0,1,2,3],   # freeze first 4 stages of the backbone
        patience  = 5,           # early stop on val mAP75 stagnation
        rect      = True,        # rectangular batches → slight speedup
        augment   = False,       # disable heavy augment ops
        project   = PROJECT,
        name      = NAME,
        exist_ok  = True,
        optimizer = "AdamW",
        lr0       = 1e-3,
        lrf       = 0.05,
        val       = True,
        save      = True,
        half      = False        # fp32 on CPU
    )
    print(f"\n✅ Done! Checkpoint & logs in {PROJECT}/{NAME}")

if __name__ == "__main__":
    main()


New https://pypi.org/project/ultralytics/8.3.127 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.78  Python-3.8.7 torch-1.13.1+cpu CPU (Intel Pentium Silver N5030 1.10GHz)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=data.yaml, epochs=20, time=None, patience=5, batch=2, imgsz=384, save=True, save_period=-1, cache=True, device=cpu, workers=4, project=runs/seg, name=cityscapes_yolov8n_seg_faster, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=[0, 1, 2, 3], multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, ag

train: Scanning C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\train\img.cache... 0 images, 2975 backgrounds, 0 corrupt: 100%|██████████| 2975/2975 [00:00<?, ?it/s]


WARNING  No labels found in C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\train\img.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.6GB RAM): 100%|██████████| 2975/2975 [00:09<00:00, 305.01it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
WARNING  'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False


...

WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.1GB RAM): 100%|██████████| 500/500 [00:01<00:00, 263.57it/s]


Plotting labels to runs\seg\cityscapes_yolov8n_seg_faster\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 384 train, 384 val
Using 0 dataloader workers
Logging results to runs\seg\cityscapes_yolov8n_seg_faster
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/20         0G          0          0     0.0191          0          0        384: 100%|██████████| 1488/1488 [30:09<00:00,  1.22s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [02:13<00:00,  1.07s/it]


RuntimeError: torch.cat(): expected a non-empty list of Tensors

## V3

In [24]:
#!/usr/bin/env python3
import os
import numpy as np
from PIL import Image

#!/usr/bin/env python3
"""
convert_masks_shifted.py

Reads your color‐coded Cityscapes masks in dataset/{train,val}/mask/*.png
and writes single‐channel index masks to dataset/{train,val}/masks/*.png,
shifting all class IDs up by +1 so that 0 becomes background and 1–10
are your semantic classes.

Usage:
    python convert_masks_shifted.py
"""

import os
import numpy as np
from PIL import Image

# 1) Define your merged palette from RGB → class ID (0–9)
PALETTE = {
    (128,  64, 128): 0,  # road_sidewalk_parking
    (244,  35, 232): 0,
    (250, 170, 160): 0,
    (250, 170,  30): 1,  # traffic
    (220, 220,   0): 1,
    (220,  20,  60): 2,  # person
    (255,   0,   0): 2,
    (  0,   0, 142): 3,  # vehicle
    (  0,   0,  70): 3,
    (  0,  60, 100): 3,
    (  0,   0,  90): 3,
    (  0,   0, 110): 3,
    (  0,  80, 100): 3,
    (  0,   0, 230): 3,
    (119,  11,  32): 3,
    (153, 153, 153): 4,  # pole
    ( 70,  70,  70): 5,  # structure
    (102, 102, 156): 5,
    (190, 153, 153): 5,
    (180, 165, 180): 5,
    (150, 100, 100): 5,
    ( 81,   0,  81): 6,  # ground
    (111,  74,   0): 7,  # dynamic
    (152, 251, 152): 8,  # terrain
    ( 70, 130, 180): 9,  # sky
}

# 2) Process both splits
for split in ("train", "val"):
    mask_in  = os.path.join("dataset", split, "mask")
    mask_out = os.path.join("dataset", split, "masks")
    os.makedirs(mask_out, exist_ok=True)

    print(f"Converting masks for split '{split}'...")
    for fname in os.listdir(mask_in):
        if not fname.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        # Load the color mask
        img_path = os.path.join(mask_in, fname)
        m = np.array(Image.open(img_path).convert("RGB"), dtype=np.uint8)

        # Prepare an empty index mask (uint8 is fine for 0–10)
        idx = np.zeros(m.shape[:2], dtype=np.uint8)

        # Map each RGB triplet to its class ID + 1
        for rgb, cid in PALETTE.items():
            mask = np.all(m == rgb, axis=-1)
            idx[mask] = cid + 1  # shift up by 1: background=0, classes=1..10

        # Save the index mask
        out_path = os.path.join(mask_out, fname)
        Image.fromarray(idx).save(out_path)

    print(f"  Saved {len(os.listdir(mask_out))} masks to '{mask_out}'\n")

print("Conversion complete!")



Converting masks for split 'train'...
  Saved 2975 masks to 'dataset\train\masks'

Converting masks for split 'val'...
  Saved 500 masks to 'dataset\val\masks'

Conversion complete!


In [ ]:
#!/usr/bin/env python3
# train_yolov8_seg_fast.py

from ultralytics import YOLO

def main():
    model = YOLO("yolov8n-seg.pt")  # segmentation‐ready model

    model.train(
        data      = "data.yaml",
        task      = "segment",       # be explicit
        epochs    = 5,
        batch     = 4,
        imgsz     = 384,
        device    = "cpu",
        workers   = 4,
        cache     = "ram",           # or "disk"
        freeze    = [0,1,2,3],
        rect      = True,
        augment   = False,
        patience  = 5,
        optimizer = "AdamW",
        lr0       = 1e-3,
        lrf       = 0.05,
        project   = "runs/seg",
        name      = "cityscapes_yolov8n_seg_fast2",
        exist_ok  = True
    )

    print("\n✅ Done! Checkpoint & logs in runs/seg/cityscapes_yolov8n_seg_fast2")

if __name__ == "__main__":
    main()


New https://pypi.org/project/ultralytics/8.3.128 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.78  Python-3.8.7 torch-1.13.1+cpu CPU (Intel Pentium Silver N5030 1.10GHz)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=data.yaml, epochs=5, time=None, patience=5, batch=4, imgsz=384, save=True, save_period=-1, cache=ram, device=cpu, workers=4, project=runs/seg, name=cityscapes_yolov8n_seg_fast2, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=[0, 1, 2, 3], multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnos

FileNotFoundError: [34m[1mtrain: [0mError loading data from C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\train_coco.json
See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [25]:
from PIL import Image
import numpy as np
m = np.array(Image.open("dataset/train/masks/train 5.png"))
print("unique values in mask:", np.unique(m))


unique values in mask: [ 0  1  2  3  4  5  6 10]


In [6]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")  # instance-seg
model.train(
    data    = "data.yaml",
    task    = "segment",
    epochs  = 20,
    batch   = 4,
    imgsz   = 384,
    device  = "cpu",
    project = "runs/seg_instances",
    name    = "cityscapes_instances",
    exist_ok=True
)


New https://pypi.org/project/ultralytics/8.3.128 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.78  Python-3.8.7 torch-1.13.1+cpu CPU (Intel Pentium Silver N5030 1.10GHz)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=4, imgsz=384, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=runs/seg_instances, name=cityscapes_instances, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnost

train: Scanning C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\train\img.cache... 2975 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2975/2975 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\MBQ\Desktop\ML Projet Test\ML Projet V3\dataset\val\img.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]

Plotting labels to runs\seg_instances\cityscapes_instances\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 384 train, 384 val
Using 0 dataloader workers
Logging results to runs\seg_instances\cityscapes_instances
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/744 [00:01<?, ?it/s]


TypeError: ERROR ❌ segment dataset incorrectly formatted or not a segment dataset.
This error can occur when incorrectly training a 'segment' model on a 'detect' dataset, i.e. 'yolo train model=yolo11n-seg.pt data=coco8.yaml'.
Verify your dataset is a correctly formatted 'segment' dataset using 'data=coco8-seg.yaml' as an example.
See https://docs.ultralytics.com/datasets/segment/ for help.

In [19]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# 1) Paths
ROOT         = r"C:/Users/MBQ/Desktop/ML Projet Test/Ptojet_ML_Yolo/dataset"
TRAIN_IMGD   = os.path.join(ROOT, "train", "img")

# 2) Your merged class palette (idx → RGB)
#    Note: YOLO-Seg saved class IDs 0–9; if you shifted +1 in masks, adjust here.
# reconstruction_palette.py

PALETTE = {
    0: (128,  64, 128),  # road_sidewalk_parking
    1: (250, 170,  30),  # traffic
    2: (220,  20,  60),  # person
    3: (  0,   0, 142),  # vehicle
    4: (153, 153, 153),  # pole
    5: ( 70,  70,  70),  # structure
    6: ( 81,   0,  81),  # ground
    7: (111,  74,   0),  # dynamic
    8: (152, 251, 152),  # terrain
    9: ( 70, 130, 180),  # sky
}

import os, cv2, numpy as np
from PIL import Image
import matplotlib.pyplot as plt

IMGDIR = r"C:/Users/MBQ/Desktop/ML Projet Test/Ptojet_ML_Yolo/dataset/train/img"
fns    = sorted([f for f in os.listdir(IMGDIR) if f.endswith(".png")])
img_fn = fns[0]
txt_fn = img_fn.replace(".png", ".txt")

# load image
orig = np.array(Image.open(os.path.join(IMGDIR, img_fn)))
h, w = orig.shape[:2]
color_mask = np.zeros_like(orig)

# draw polygons
with open(os.path.join(IMGDIR, txt_fn)) as f:
    for line in f:
        parts = line.strip().split()
        cls   = int(parts[0])
        if len(parts) <= 5:
            continue
        coords = np.array(list(map(float, parts[5:])))
        xs = (coords[0::2] * w).astype(int)
        ys = (coords[1::2] * h).astype(int)
        pts = np.stack((xs, ys), axis=1).reshape(-1,1,2)
        cv2.fillPoly(color_mask, [pts], PALETTE[cls])

overlay = cv2.addWeighted(orig, 0.6, color_mask, 0.4, 0)

plt.figure(figsize=(12,4))
for i,(im,t) in enumerate(zip((orig,color_mask,overlay),
                             ("Original","Mask","Overlay"))):
    plt.subplot(1,3,i+1)
    plt.imshow(cv2.cvtColor(im,cv2.COLOR_BGR2RGB))
    plt.title(t); plt.axis("off")
plt.show()
Image.fromarray(color_mask).save("mask_color.png")
print("Saved color mask to mask_color.png")


<Figure size 640x480 with 0 Axes>

<Figure size 1200x400 with 3 Axes>

Saved color mask to mask_color.png
